In [6]:
!pip uninstall torch torchvision torchtext torchaudio fastai
!pip3 install --pre torch==2.0.0.dev20230301 --index-url https://download.pytorch.org/whl/nightly/cpu
import logging
from typing import List
import traceback as tb

import torch
print(torch.__version__)
import torch._dynamo
import torch._dynamo.config

Looking in indexes: https://download.pytorch.org/whl/nightly/cpu, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.4 MB/s eta 0:00:00
2.0.0.dev20230202+cpu


In [2]:
torch._dynamo.config.log_level = logging.INFO
torch._dynamo.config.output_code = True
torch._dynamo.config.verbose = False

In [7]:
## GUARDS OVERVIEW
def my_compiler(gm: torch.fx.GraphModule, example_inputs: List[torch.Tensor]):
    print("my_compiler() called with FX graph:")
    gm.graph.print_tabular()
    return gm.forward  # return a python callable
torch._dynamo.reset()
@torch._dynamo.optimize(my_compiler)
def toy_example(a, b):
    x = a / (torch.abs(a) + 1)
    if b.sum() < 0:
        b = b * -1
    return x * b
for _ in range(100):
    toy_example(torch.randn(10), torch.randn(10))

[2023-03-03 06:28:46,644] torch._dynamo.symbolic_convert: [INFO] Step 1: torchdynamo start tracing toy_example
[2023-03-03 06:28:46,668] torch._dynamo.output_graph: [INFO] Step 2: calling compiler function my_compiler
[2023-03-03 06:28:46,676] torch._dynamo.output_graph: [INFO] Step 2: done compiler function my_compiler
[2023-03-03 06:28:46,679] torch._dynamo.output_graph: [INFO] TRACED GRAPH
 __compiled_fn_5 <eval_with_key>.7 opcode         name     target                                                  args              kwargs
-------------  -------  ------------------------------------------------------  ----------------  --------
placeholder    a        a                                                       ()                {}
placeholder    b        b                                                       ()                {}
call_function  abs_1    <built-in method abs of type object at 0x7fbab3f9d940>  (a,)              {}
call_function  add      <built-in function add>       

my_compiler() called with FX graph:
opcode         name     target                                                  args              kwargs
-------------  -------  ------------------------------------------------------  ----------------  --------
placeholder    a        a                                                       ()                {}
placeholder    b        b                                                       ()                {}
call_function  abs_1    <built-in method abs of type object at 0x7fbab3f9d940>  (a,)              {}
call_function  add      <built-in function add>                                 (abs_1, 1)        {}
call_function  truediv  <built-in function truediv>                             (a, add)          {}
call_method    sum_1    sum                                                     (b,)              {}
call_function  lt       <built-in function lt>                                  (sum_1, 0)        {}
output         output   output               

In [ ]:
## CUSTOM BACKENDS
def my_compiler(gm: torch.fx.GraphModule, example_inputs: List[torch.Tensor]):
    print("my_compiler() called with FX graph:")
    gm.graph.print_tabular()
    return gm.forward  # return a python callable
torch._dynamo.reset()
@torch._dynamo.optimize(my_compiler)
def fn(x, y):
    a = torch.cos(x)
    b = torch.sin(y)
    return a + b
fn(torch.randn(10), torch.randn(10))

In [ ]:
def my_compiler(gm: torch.fx.GraphModule, example_inputs: List[torch.Tensor]):
    print("my_compiler() called with FX graph:")
    gm.graph.print_tabular()
    return gm.forward  # return a python callable
torch._dynamo.reset()

# nn.Module
class MockModule(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.relu = torch.nn.ReLU()
    def forward(self, x):
        return self.relu(torch.cos(x))
mod = MockModule()
torch._dynamo.reset()
optimized_mod = torch._dynamo.optimize(my_compiler)(mod)
optimized_mod(torch.randn(10))

In [12]:
# control flow
@torch._dynamo.optimize(my_compiler)
def toy_example(a, b):
    x = a / (torch.abs(a) + 1)
    if b.sum() < 0:
        b = b * -1
    return x * b
for _ in range(100):
    toy_example(torch.randn(10), torch.randn(10))
try:
    torch._dynamo.export(toy_example, torch.randn(10), torch.randn(10))
except:
    tb.print_exc()

explanation, out_guards, graphs, ops_per_graph, break_reasons, explanation_verbose = torch._dynamo.explain(
    toy_example, torch.randn(10), torch.randn(10)
)

print(f'explanation: {explanation}')

print(graphs[0].print_readable())
print(graphs[1].print_readable())

Traceback (most recent call last):
  File "<ipython-input-12-9e8b6974b0ee>", line 15, in <module>
    torch._dynamo.export(toy_example, torch.randn(10), torch.randn(10))
  File "/usr/local/lib/python3.8/dist-packages/torch/_dynamo/eval_frame.py", line 602, in export
  File "/usr/local/lib/python3.8/dist-packages/torch/_dynamo/eval_frame.py", line 209, in _fn
    if (
TypeError: toy_example() missing 1 required positional argument: 'c'
[2023-03-03 06:41:51,634] torch._dynamo.symbolic_convert: [INFO] Step 1: torchdynamo start tracing toy_example
[2023-03-03 06:41:51,646] torch._dynamo.output_graph: [INFO] Step 2: calling compiler function dynamo_graph_accumulating_compiler
[2023-03-03 06:41:51,647] torch._dynamo.output_graph: [INFO] Step 2: done compiler function dynamo_graph_accumulating_compiler
[2023-03-03 06:41:51,651] torch._dynamo.output_graph: [INFO] TRACED GRAPH
 __compiled_fn_82 <eval_with_key>.77 opcode         name     target                                                  ar

graph len: 4


In [ ]:
## Pytorch con2023 example
def my_fn(gm, inputs):
    gm.graph.print_tabular()
    # gm.print_readable()
    return gm.forward

def foo(x, y):
    a = x * y
    print("break!")
    b = x * y
    torch._dynamo.graph_break()
    return a + b
torch._dynamo.reset()
torch._dynamo.optimize(my_fn)(foo)(
    torch.randn([2, 2]),
    torch.randn([2, 2]))